In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

##### Attention:
For this lab, there are no local answers for the questions in learntools. However, there are some assertions for the first 4 questions. You should pass all of the assertions before submitting to the autograding.

Some of the questions use mathematical symbols and equations to depict the scenario. These are just some basic "formal" definitions and you should understand them. In case you haven't understood them, read again and give yourself sometime to ponder. If there is any error or details in the questions that heavily confuse you, please post your concerns on Google Classroom. 

## Questions

### Q1: Given a function $f(x) = x^2 + 3x + 8$, return the gradient of $x$ when $x=2.0$ using pytorch autograd. The gradient tensor should have type torch.float32 and dim=0.

In [2]:
def exercise_1() -> torch.Tensor:
    x = torch.tensor(2.0,dtype=torch.float32,requires_grad=True)
    f = x**2 +3*x +8
    f.backward()
    return x.grad

ex1_sol = exercise_1()
assert torch.equal(ex1_sol, torch.tensor(7.0))

### Q2: A typical layer in a Neural Network would look like $f(x) = w_1 x_1 + w_2 x_2 + w_3 x_3 + ... + b$, where $w_j$ is the j-th weight of the function and $b$ is its bias. This function outputs a result for a multi-dimension input, where $x_j$ is the j-th feature of the input. Implement this function, return the gradients of each weight for a given input `x` and the output using `tuple` with the formar `(output, weight_grads_tensor)`. The weights should be randomized using `torch.randn()` and the bias must be set to $5.0$. The input used by the autograder will be the same size as the one given below.

In [3]:
x = torch.tensor([1, 2, 3, 4, 5])

def exercise_2(x) -> tuple:
    # all the output and gradients are tensors

    torch.manual_seed(0) # must set seed=0 before random your weight
    x = x.to(torch.float32)
    weights = torch.randn(x.shape[0],dtype=torch.float32, requires_grad=True)
    bias = torch.tensor(5.0,dtype=torch.float32, requires_grad=True)
    output = torch.dot(weights,x) + bias
    output.backward()
    weight_grads_tensor = weights.grad
    return(output,weight_grads_tensor)
    # you must return in this format
    # return (output, weight_grads_tensor)

ex2_sol = exercise_2(x)

assert (ex2_sol[0] - torch.tensor(-3.7311) < 1e-4).item()
assert torch.equal(ex2_sol[1], torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32))

### Q3: Suppose each feature $x_j$ needs a separate function $f_j(x_j)$, where $f_j(x_j) = w_j x_j + b_j$. In other words, each feature requires a function, which has a separate set of 1 weight and 1 bias. Therefore, the output $y$ of your function $G(x)=[f_1(x_1), f_2(x_2), ..., f_n(x_n)]$ should be $y = [(w_1 x_1 + b_1), (w_2 x_2 + b_2), ..., (w_n x_n + b_n)]$ (**note:** these are matrices written lazily with markdown). Implement the function $G(x)$, return the output and the gradient sets for each function for a given `x`. The weights should be randomized using `torch.randn()`, and the bias is the same as $j$ of $f_j()$ (for example: $b_1 = 1.0$, $b_2 = 2.0$). The input used by the autograder will be the same size as the one given below.

In [4]:
x = torch.tensor([1, 2, 3, 4, 5])

def exercise_3(x) -> tuple:
     # all the output and gradients are tensors
    
    torch.manual_seed(0) # must set seed=0 before random your weight

    # Initialize weights with requires_grad=True
    weights = torch.randn(x.size(0), dtype=torch.float32, requires_grad=True)
    
    # Biases: b_j = j
    biases = torch.tensor([float(i+1) for i in range(x.size(0))], dtype=torch.float32)
    
    # Compute outputs
    y = weights * x + biases
    
    # Dummy scalar loss to backpropagate from (sum is common trick)
    loss = y.sum()
    loss.backward()
    
    # Get gradients of weights
    weight_grads = weights.grad
    
    return (y, weight_grads)
    # you must return in this format
    # return (output, weight_grads_tensor)

ex3_sol = exercise_3(x)
assert torch.all((ex3_sol[0] - torch.tensor([ 2.5410,  1.4131, -3.5364,  6.2737, -0.4226 ], dtype=torch.float32)) < 5e-4).item()
assert torch.equal(ex3_sol[1], torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32))
# q3.check()

### Q4: Using the function $f(x)$ similar to the one in question 2, implement the function with random weights and bias (must use `torch.manual_seed(0)` before each initialization). Calculate the Mean Squared Error between the `target` and your function prediction on the `data` as a loss metric, then return a tuple containing the gradients of your function weights and bias based on the calculated loss.

In [5]:
data = torch.tensor([2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0])
target = torch.tensor([2.0])

def exercise_4(data, target) -> tuple:
    # both gradients are tensors
    torch.manual_seed(0)  # Set seed before initializing weights
    data = data.to(torch.float32)
    
    weights = torch.randn(data.shape[0], dtype=torch.float32, requires_grad=True)
    bias = torch.randn(1, dtype=torch.float32, requires_grad=True)
    
    output = torch.dot(weights, data) + bias  # scalar output
    loss = torch.mean((output - target) ** 2)  # MSE loss
    
    loss.backward()  # compute gradients
    
    return (weights.grad, bias.grad)
    # you must return in this format
    # return (weight_grads_tensor, bias_grads_tensor)

ex4_sol = exercise_4(data, target)
assert ex4_sol[0].dim() == 1 and ex4_sol[1].dim() == 1, print("All tensors must have dim == 1")
assert torch.all((ex4_sol[0] - torch.tensor([-85.1518, -127.7277, -170.3036, -212.8795, -255.4553, -298.0312, -340.6071, -383.1830, -425.7589, -468.3348], dtype=torch.float32)) < 5e-4)
assert torch.all((ex4_sol[1] - torch.tensor([-42.5759], dtype=torch.float32)) < 5e-4)

### Q5: This question worth 40% the points of this lab. Your solution for this question will be graded manually by the TAs, hence you will be informed later for the total points for this lab after the deadline has been met.

**Note**: This question will not focus on getting the right results as your implementation is more important. Therefore there will be no local answer or autograding.

#### Create a small neural network consisting 2 Linear layers, each followed by a Sigmoid activation function. The template code has been provided below including the return format (`x` should be replaced by the final `tensor` after the forward pass). Your code must use layers and functions provided by `pytorch`. Initialize all necessary components that can be used during the training phase including a Binary Cross Entropy Loss function and a Gradient Descent optimizer. Train your model for 2 epochs with a proper train loop, then make a prediction on a given `val_x` and return your prediction.

**Note**: the first Linear Layer can have any `in_features` and `out_features`, but the last Linear Layer must have `out_features=2`. Your model will consume the whole input for each epoch, so you don't need to separate the input into batches.

*Context*: This part is not necessary for your implementation, but it could help you to understand the work that your network is simulating. The `train_x` consists grades for 5 subjects of 20 students, with the minimum grade is 1 and the maximum is 10. The problem is a classification task, when your model predicts if a student is good or bad based on their 5 grades. That is the reason why the final layer must have 2 output features, which correspond to the 2 classes "good" and "bad".

In [6]:
train_x = torch.randint(1, 10, (20, 5), dtype=torch.float32) # input
train_y = (train_x.sum(1) > 30).float() # target
val_x = torch.randint(3, 10, (10, 5), dtype=torch.float32)

def exercise_5(train_x, train_y, val_x) -> torch.Tensor:

    class MyNetwork(nn.Module): #define NN
        def __init__(self):
            super().__init__()
            # https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
            self.layer1 = nn.Linear(5, 4)  # 5 inputs -> 4 outputs (1st Linear Layer)
            self.layer2 = nn.Linear(4, 2)  # 4 inputs -> 2 class outputs (2nd Linear Layer)
            self.activation = nn.Sigmoid() # https://docs.pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html#torch.nn.Sigmoid

        def forward(self, x):
            x = self.activation(self.layer1(x)) #Apply 1st & 2nd layer
            x = self.activation(self.layer2(x))
            return x

    model = MyNetwork()

    loss_fn = nn.BCELoss() # https://docs.pytorch.org/docs/stable/generated/torch.nn.BCELoss.html#torch.nn.BCELoss
    # Binary Cross Entropy Loss for multi-label (2-class) classification
    # Inputs and targets must both be in [0, 1]

    optimizer = optim.SGD(model.parameters(), lr=0.01) # https://docs.pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD
    # Stochastic Gradient Descent optimizer

    train_y_one_hot = torch.stack([1 - train_y, train_y], dim=1) # https://docs.pytorch.org/docs/stable/generated/torch.stack.html#torch.stack
    # Convert 1D target [batch_size] -> 2D one-hot [batch_size, 2]
    # Class 0: "not good" (sum <= 30), Class 1: "good" (sum > 30)

    for epoch in range(2):
        optimizer.zero_grad()
        output = model(train_x)
        loss = loss_fn(output, train_y_one_hot)
        loss.backward()
        optimizer.step()

        # Clear Prev gradients
        # Fwd Pass
        # Compute Loss
        # Backpropagate
        # Update weights

    with torch.no_grad(): # https://docs.pytorch.org/docs/stable/generated/torch.no_grad.html#torch.no_grad
        val_pred = model(val_x) # get prediction for validation input

    return val_pred

ex5_sol = exercise_5(train_x=train_x, train_y=train_y, val_x=val_x)